In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm
import spacy
import pandas as pd


import sys
sys.path.append("../../")
import os
from src.functional import free_gpu_cache
from src.utils import env_utils

from src.models import load_LM
import torch
from nnsight import LanguageModel
from src.utils import env_utils

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:


# model_key = "meta-llama/Meta-Llama-3-8B-Instruct"
model_key = "meta-llama/Meta-Llama-3-70B-Instruct"

lm = load_LM(
    model_key=model_key,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 30/30 [01:15<00:00,  2.51s/it]

loaded /home/local_arnab/Codes/00_MODEL/meta-llama/Meta-Llama-3-70B-Instruct | size: 36650.535 MB


In [4]:
from src.dataset import DatasetV2, SampleV2

with open (os.path.join(env_utils.DEFAULT_DATA_DIR, "dataset_v2.json"), "r") as f:
    dataset_dict = json.load(f)

dataset = DatasetV2.from_dict(dataset_dict)
print(len(dataset))

5561


In [5]:
from src.functional import get_hs, find_token_range, prepare_input, logit_lens
def collect_actor_latent_in_question(
    lm: LanguageModel,
    question: str,
    actor: str,
    layers: list = list(range(7, 28)),
    layer_name_format: str = "model.layers.{}"
) -> dict[int: torch.Tensor]:
    inputs = prepare_input(prompts=question, tokenizer=lm, return_offsets_mapping=True)
    action_last_range = find_token_range(
        string=question,
        substring=actor,
        occurrence=-1,
        offset_mapping=inputs["offset_mapping"][0],
    )
    actor_last_idx = action_last_range[1] - 1
    inputs.pop("offset_mapping")
    print(actor_last_idx, " >> ", lm.tokenizer.decode(inputs["input_ids"][0][actor_last_idx]))

    
    last_loc = (layer_name_format.format(lm.config.num_hidden_layers-1), -1)
    locations = [(layer_name_format.format(i), actor_last_idx) for i in layers] + [last_loc]

    hs = get_hs(
        lm=lm,
        input=inputs,
        locations=locations,
        return_dict=True
    )

    predicted_ans = logit_lens(lm = lm, h = hs[last_loc], k = 2)[0]
    print(f"{predicted_ans=}")

    return hs

In [6]:
idx = 25
question, answer = dataset.__getitem__(idx, set_ans="yes")
# question = question.replace("Answer:", "Answer (yes/no):")
actor = dataset.samples[idx].actor
print(question)
print(answer)

hs = collect_actor_latent_in_question(
    lm=lm,
    question=question,
    actor=actor,
    layers=list(range(7, 28)),
)

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose "yes" or "no" after the "Answer (yes/no):" tag.

Story: Rob works in a busy restaurant. A customer asks Rob for aquavit. Rob grabs an opaque pitcher and fills it with aquavit. A coworker, who didn't hear the customer's request, swaps the aquavit in the pitcher with ouzo while Rob is attending to another task. Rob does not see the coworker swapping the contents in the pitcher and also can't see what is in it without opening it.

Question: Does Rob believe that there is aquavit in the pitcher?
Answer:
yes
147  >>   Rob


/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


predicted_ans=PredictedToken(token=' yes', prob=0.5722293257713318, logit=21.953125, token_id=10035)


## Parse cached states

In [4]:
from scripts.collect_binding_id_states import CachedBindingIDState

cache_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "binding_id_states_split",
    # lm.config._name_or_path.split("/")[-1],
    "Meta-Llama-3-70B-Instruct"

)

all_cached_states: list[CachedBindingIDState] = []
for file_name in tqdm(os.listdir(cache_path)):
    if not file_name.endswith(".json"):
        continue
    with open(os.path.join(cache_path, file_name), "r") as f:
        loaded_dict = json.load(f)
        # cached_state = CachedBindingIDState.from_dict(loaded_dict)
        # all_cached_states.append(cached_state)
        all_cached_states.append(loaded_dict)

  0%|          | 0/5561 [00:00<?, ?it/s]

100%|██████████| 5561/5561 [04:10<00:00, 22.19it/s]


In [7]:
def check_actor_position(keys):
    positions = list(set([int(k.split("_<>_")[-1]) for k in keys]))
    return min(positions)

free_gpu_cache()

layers = list(range(7, 28))
layer_name_format = "model.layers.{}"
Delta = {
    layer_name_format.format(i): [] for i in layers
}

for cached_state in tqdm(all_cached_states):
    actor_position = check_actor_position(cached_state["context_informed_actor"].keys())
    for layer in Delta:
        d = (
                torch.Tensor(cached_state["context_informed_actor"][f"{layer}_<>_{actor_position}"]) -
                torch.Tensor(cached_state["context_unaware_actor"][f"{layer}_<>_{actor_position}"])
        )
        Delta[layer].append(d) 

free_gpu_cache()

100%|██████████| 5561/5561 [01:07<00:00, 82.59it/s]


In [8]:
for layer in Delta:
    Delta[layer] = torch.stack(Delta[layer])

Delta[layer].shape

torch.Size([5561, 8192])

In [9]:
projection_matrices = {}

low_rank = 100
for layer in Delta:
    print(layer)
    U, S, Vh = torch.linalg.svd(Delta[layer].cuda())
    projection_matrices[layer] = Vh[: low_rank]

    free_gpu_cache()

model.layers.7
model.layers.8
model.layers.9
model.layers.10
model.layers.11
model.layers.12
model.layers.13
model.layers.14
model.layers.15
model.layers.16
model.layers.17
model.layers.18
model.layers.19
model.layers.20
model.layers.21
model.layers.22
model.layers.23
model.layers.24
model.layers.25
model.layers.26
model.layers.27


In [10]:
import numpy as np
np.savez_compressed(
    os.path.join(env_utils.DEFAULT_RESULTS_DIR, "projection_matrices.npz"),
    **{
        layer: projection_matrices[layer].cpu().numpy() for layer in projection_matrices
    }
)

## Check Causality

In [8]:
import numpy as np

loaded_projections = np.load(
    os.path.join(env_utils.DEFAULT_RESULTS_DIR, "projection_matrices.npz")
)

V_projections = {}
for layer in loaded_projections:
    V_projections[layer] = torch.Tensor(loaded_projections[layer]).cuda()
    print(layer, V_projections[layer].shape)

model.layers.7 torch.Size([100, 8192])
model.layers.8 torch.Size([100, 8192])
model.layers.9 torch.Size([100, 8192])
model.layers.10 torch.Size([100, 8192])
model.layers.11 torch.Size([100, 8192])
model.layers.12 torch.Size([100, 8192])
model.layers.13 torch.Size([100, 8192])
model.layers.14 torch.Size([100, 8192])
model.layers.15 torch.Size([100, 8192])
model.layers.16 torch.Size([100, 8192])
model.layers.17 torch.Size([100, 8192])
model.layers.18 torch.Size([100, 8192])
model.layers.19 torch.Size([100, 8192])
model.layers.20 torch.Size([100, 8192])
model.layers.21 torch.Size([100, 8192])
model.layers.22 torch.Size([100, 8192])
model.layers.23 torch.Size([100, 8192])
model.layers.24 torch.Size([100, 8192])
model.layers.25 torch.Size([100, 8192])
model.layers.26 torch.Size([100, 8192])
model.layers.27 torch.Size([100, 8192])


In [88]:
from src.functional import predict_next_token

idx = 0
question, answer = dataset.__getitem__(idx, set_ans="yes")
actor = dataset.samples[idx].actor
print(question)
print(answer)

predict_next_token(lm=lm, input=question)

Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose "yes" or "no" after the "Answer (yes/no):" tag.

Story: Eve works in a busy restaurant. A customer asks Eve for whiskey. Eve grabs an opaque pitcher and fills it with whiskey. A coworker, who didn't hear the customer's request, swaps the whiskey in the pitcher with brandy while Eve is attending to another task. Eve does not see the coworker swapping the contents in the pitcher and also can't see what is in it without opening it.

Question: Does Eve believe that there is whiskey in the pitcher?
Answer:
yes


[PredictedToken(token=' yes', prob=0.5230958461761475, logit=21.34375, token_id=10035),
 PredictedToken(token=' YES', prob=0.27999305725097656, logit=20.71875, token_id=14410),
 PredictedToken(token=' Yes', prob=0.10627337545156479, logit=19.75, token_id=7566),
 PredictedToken(token=' (', prob=0.05513389781117439, logit=19.09375, token_id=320),
 PredictedToken(token=' **', prob=0.013095449656248093, logit=17.65625, token_id=3146)]

In [89]:
unaligned_question, answer = dataset.__getitem__(idx, set_ans="yes", unaligned_actor="Adam")
print(unaligned_question)

predict_next_token(lm=lm, input=unaligned_question, interested_tokens=[10035])

Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose "yes" or "no" after the "Answer (yes/no):" tag.

Story: Adam works in a busy restaurant. A customer asks Adam for whiskey. Adam grabs an opaque pitcher and fills it with whiskey. A coworker, who didn't hear the customer's request, swaps the whiskey in the pitcher with brandy while Adam is attending to another task. Adam does not see the coworker swapping the contents in the pitcher and also can't see what is in it without opening it.

Question: Does Eve believe that there is whiskey in the pitcher?
Answer:


([PredictedToken(token=' No', prob=0.20485912263393402, logit=16.359375, token_id=2360),
  PredictedToken(token=' There', prob=0.19244733452796936, logit=16.296875, token_id=2684),
  PredictedToken(token=' (', prob=0.09019824862480164, logit=15.5390625, token_id=320),
  PredictedToken(token=' Not', prob=0.0789802223443985, logit=15.40625, token_id=2876),
  PredictedToken(token=' no', prob=0.07135268300771713, logit=15.3046875, token_id=912)],
 {10035: (20,
   PredictedToken(token=' yes', prob=0.004386679269373417, logit=12.515625, token_id=10035))})

In [90]:
inputs = prepare_input(prompts=question, tokenizer=lm, return_offsets_mapping=True)
action_last_range = find_token_range(
    string=question,
    substring=actor,
    occurrence=-1,
    offset_mapping=inputs["offset_mapping"][0],
)
actor_last_idx = action_last_range[1] - 1
inputs.pop("offset_mapping")

print(actor_last_idx)

144


In [101]:
layer_name_format = "model.layers.{}"

s_a = collect_actor_latent_in_question(
    lm = lm, 
    question = question,
    actor = actor
)

s_u = collect_actor_latent_in_question(
    lm = lm,
    question = unaligned_question,
    actor = actor
)

print(f"{s_a[(layer_name_format.format(15), actor_last_idx)].shape=}")
print(f"{s_u[(layer_name_format.format(15), actor_last_idx)].shape=}")

144  >>   Eve
predicted_ans=PredictedToken(token=' yes', prob=0.5230958461761475, logit=21.34375, token_id=10035)
144  >>   Eve
predicted_ans=PredictedToken(token=' No', prob=0.20485912263393402, logit=16.359375, token_id=2360)
s_a[(layer_name_format.format(15), actor_last_idx)].shape=torch.Size([8192])
s_u[(layer_name_format.format(15), actor_last_idx)].shape=torch.Size([8192])


In [102]:
def patch_binding_ID(
    s_orig: torch.Tensor,
    s_other: torch.Tensor,
    Proj: torch.Tensor,
    rank: int = 10
):
    s_orig = s_orig.to(torch.float32)
    s_other = s_other.to(torch.float32)
    Proj = Proj.to(torch.float32)

    Proj_r = Proj[:rank].to(lm.device)
    Proj_r = Proj_r.T @ Proj_r
    return (
        (torch.eye(Proj_r.shape[0]).to(lm.device) - Proj_r) @ s_orig # remove patching ID from s_orig
        + Proj_r @ s_other # add patching ID from s_other
    )

In [103]:
from src.functional import PatchSpec, free_gpu_cache

free_gpu_cache()

patches = []
for layer_idx in range(7, 28):
    layer = layer_name_format.format(layer_idx)
    location = (layer, actor_last_idx)
    patches.append(PatchSpec(
        location=location,
        patch=patch_binding_ID(
            s_orig=s_a[location],
            s_other=s_u[location],
            Proj=V_projections[layer],
            rank=1
        )
    ))

In [106]:
predict_next_token(
    lm=lm, 
    # input=unaligned_question,
    input=question,
    patches=patches,
    interested_tokens=[10035]
)

([PredictedToken(token=' There', prob=0.3007343113422394, logit=16.40625, token_id=2684),
  PredictedToken(token=' No', prob=0.14205683767795563, logit=15.65625, token_id=2360),
  PredictedToken(token=' (', prob=0.11414587497711182, logit=15.4375, token_id=320),
  PredictedToken(token=' NO', prob=0.05968255549669266, logit=14.7890625, token_id=5782),
  PredictedToken(token=' no', prob=0.049478575587272644, logit=14.6015625, token_id=912)],
 {10035: (19,
   PredictedToken(token=' yes', prob=0.004937465768307447, logit=12.296875, token_id=10035))})

In [95]:
def swap_objs(story, obj_1, obj_2):
    story = story.replace(obj_1, "<obj_placeholder>")
    story = story.replace(obj_2, obj_1)
    story = story.replace("<obj_placeholder>", obj_2)
    return story

sample = dataset.samples[idx]
sample_obj_swapped = SampleV2(
    story=swap_objs(sample.story, sample.obj_belief, sample.obj_true),
    actor=sample.actor,
    obj_belief=sample.obj_true,
    obj_true=sample.obj_belief,
    container=sample.container,
)

temp_d = DatasetV2(samples=[sample, sample_obj_swapped])

In [96]:
ques_1, ans_1 = temp_d.__getitem__(0, set_ans="no")
print(ques_1)
print(ans_1)

print("\n\n")
ques_2, ans_2 = temp_d.__getitem__(1, set_ans="yes")
print(ques_2)
print(ans_2)

Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose "yes" or "no" after the "Answer (yes/no):" tag.

Story: Eve works in a busy restaurant. A customer asks Eve for whiskey. Eve grabs an opaque pitcher and fills it with whiskey. A coworker, who didn't hear the customer's request, swaps the whiskey in the pitcher with brandy while Eve is attending to another task. Eve does not see the coworker swapping the contents in the pitcher and also can't see what is in it without opening it.

Question: Does Eve believe that there is brandy in the pitcher?
Answer:
no



Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose "yes" or "no" after the "Answer (yes/no):" tag.


In [97]:
s_1 = collect_actor_latent_in_question(
    lm = lm, 
    question = ques_1,
    actor = sample.actor
)

s_2 = collect_actor_latent_in_question(
    lm = lm,
    question = ques_2,
    actor = sample.actor
)

144  >>   Eve
predicted_ans=PredictedToken(token=' no', prob=0.47336092591285706, logit=20.40625, token_id=912)
146  >>   Eve
predicted_ans=PredictedToken(token=' yes', prob=0.5370039343833923, logit=21.4375, token_id=10035)


In [98]:
predict_next_token(
    lm=lm, 
    input=ques_1,
    interested_tokens=[10035]
)

([PredictedToken(token=' no', prob=0.47336092591285706, logit=20.40625, token_id=912),
  PredictedToken(token=' No', prob=0.245576411485672, logit=19.75, token_id=2360),
  PredictedToken(token=' NO', prob=0.11600209027528763, logit=19.0, token_id=5782),
  PredictedToken(token=' (', prob=0.07373541593551636, logit=18.546875, token_id=320),
  PredictedToken(token=' **', prob=0.046869080513715744, logit=18.09375, token_id=3146)],
 {10035: (14,
   PredictedToken(token=' yes', prob=0.0005288677639327943, logit=13.609375, token_id=10035))})

In [100]:
idx_1 = 144
idx_2 = 146

patches = []
for layer_idx in range(7, 28):
    layer = layer_name_format.format(layer_idx)
    location_1 = (layer, idx_1)
    location_2 = (layer, idx_2)
    patches.append(PatchSpec(
        location=location,
        patch=patch_binding_ID(
            s_orig=s_1[location_1],
            s_other=s_2[location_2],
            Proj=V_projections[layer],
            rank=50
        )
    ))

predict_next_token(
    lm=lm, 
    input=ques_1,
    patches=patches,
    interested_tokens=[10035]
)

([PredictedToken(token=' no', prob=0.4730239808559418, logit=20.390625, token_id=912),
  PredictedToken(token=' No', prob=0.24540160596370697, logit=19.734375, token_id=2360),
  PredictedToken(token=' NO', prob=0.11591951549053192, logit=18.984375, token_id=5782),
  PredictedToken(token=' (', prob=0.07368292659521103, logit=18.53125, token_id=320),
  PredictedToken(token=' **', prob=0.04683571681380272, logit=18.078125, token_id=3146)],
 {10035: (14,
   PredictedToken(token=' yes', prob=0.0005410240846686065, logit=13.6171875, token_id=10035))})